In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.optim as optim

In [2]:
num_epochs = 8
num_classes = 2
batch_size = 128
learning_rate = 0.002
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
labels = pd.read_csv('/kaggle//input/histopathologic-cancer-detection/train_labels.csv')
sub = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/sample_submission.csv')
train_path = '/kaggle/input/histopathologic-cancer-detection/train/'
test_path = '/kaggle/input/histopathologic-cancer-detection/test/'

In [4]:
train, val = train_test_split(labels, stratify=labels.label, test_size=0.1)
len(train), len(val)

(198022, 22003)

In [5]:
class CreateDataset(Dataset):
    def __init__(self,df_data,data_dir = './',transform=None):
        super().__init__()
        self.df=df_data.values
        self.data_dir=data_dir
        self.transform=transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        img_name,label=self.df[index]
        img_path=os.path.join(self.data_dir,img_name+'.tif')
        image= cv2.imread(img_path)
        if self.transform is not None:
            image=self.transform(image)
        return image,label    
        

In [6]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = CreateDataset(df_data=train, data_dir=train_path, transform=trans_train)
dataset_valid = CreateDataset(df_data=val, data_dir=train_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [7]:
len(loader_valid)

344

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.bn2 = nn.BatchNorm2d(64)                                                       # Neurony
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(8)
        self.fc = nn.Linear(512 * 1 * 1, 2) # Flatten
        
        
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) 
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))         # Aktywacja
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        x = x.view(-1, 512 * 1 * 1) 
        x = self.fc(x)
        return x

In [9]:
train_on_gpu=torch.cuda.is_available()

if not train_on_gpu:
    print('Niet')
else:
    print('Da')
    

model = CNN().to(device)
print(model)

if train_on_gpu: model.cuda()
    

Da
CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg): AvgPool2d(kernel_size=8, stride=8, padding=0)
  (fc): Linear(

In [10]:
# Optimizer i CEL
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [11]:
def train(net):
    total_step = len(loader_train)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loader_train):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

train(model)

Epoch [1/8], Step [100/1548], Loss: 0.4752
Epoch [1/8], Step [200/1548], Loss: 0.4125
Epoch [1/8], Step [300/1548], Loss: 0.3555
Epoch [1/8], Step [400/1548], Loss: 0.3154
Epoch [1/8], Step [500/1548], Loss: 0.3327
Epoch [1/8], Step [600/1548], Loss: 0.5144
Epoch [1/8], Step [700/1548], Loss: 0.3286
Epoch [1/8], Step [800/1548], Loss: 0.3829
Epoch [1/8], Step [900/1548], Loss: 0.3244
Epoch [1/8], Step [1000/1548], Loss: 0.3016
Epoch [1/8], Step [1100/1548], Loss: 0.2735
Epoch [1/8], Step [1200/1548], Loss: 0.2288
Epoch [1/8], Step [1300/1548], Loss: 0.2283
Epoch [1/8], Step [1400/1548], Loss: 0.4031
Epoch [1/8], Step [1500/1548], Loss: 0.2696
Epoch [2/8], Step [100/1548], Loss: 0.2658
Epoch [2/8], Step [200/1548], Loss: 0.2479
Epoch [2/8], Step [300/1548], Loss: 0.2730
Epoch [2/8], Step [400/1548], Loss: 0.2257
Epoch [2/8], Step [500/1548], Loss: 0.1953
Epoch [2/8], Step [600/1548], Loss: 0.2645
Epoch [2/8], Step [700/1548], Loss: 0.2454
Epoch [2/8], Step [800/1548], Loss: 0.1387
Epoch

In [12]:
def test(net):
    
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loader_valid:
            images = images.cuda()
            labels = labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

test(model)

Test Accuracy of the model on the test images: 94.30532200154525 %


In [13]:
#Submission
model.eval()

preds= []

for batch_i,(data,target) in enumerate(loader_valid):
    data,target=data.cuda(),target.cuda()
    output = model(data)
    
    
    pr=output.detach().cpu().numpy()
    for i in pr:
        preds.append(i)
        
sample_sub['label'] = preds


NameError: name 'sample_sub' is not defined

In [14]:
for i in range(len(sample_sub)):
    sample_sub.label[i] = np.float(sample_sub.label[i])
    

NameError: name 'sample_sub' is not defined

In [15]:
sample_sub.to_csv('submission.csv', index=False)
sample_sub.head()

NameError: name 'sample_sub' is not defined